In [ ]:
from requests import get, post
from sys import exit
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

HOST = 'https://api.chartmetric.com'
TOKEN = os.getenv('REFRESH_TOKEN')
res = post(f'{HOST}/api/token', json={"refreshtoken": TOKEN})
if res.status_code != 200:
    print(f'ERROR: received a {res.status_code} instead of 200 from /api/token')
    exit(1)

access_token = res.json()['token']
def Get(uri):
    return get(f'{HOST}{uri}', headers={'Authorization': f'Bearer {access_token}'})

tracks = []
for i in range(0, 10000, 100):
    res = Get(f'/api/track/list/filter?limit=100&offset={i}')
    if res.status_code != 200:
        print(f'ERROR: received a {res.status_code} instead of 200 from /api/track/list/filter')
        exit(1)
        
    objs = res.json()['obj']

    for obj in objs:
        track = {
            'id': obj['cm_track'],
            'track_name': obj['name'],
            'album_name': obj['album'][0]['name'],
            'release_date': obj['album'][0]['release_date'],
            'artist': [artist['name'] for artist in obj['artist']],
            'genre': obj['genre'],
            'explicit': obj['explicit'],
            'score': obj['score'],
        }
        
        for key in obj['latest'].keys():
            if key == 'rank':
                continue
            track[key] = obj['latest'][key]
        tracks.append(track)
    
tracks_df = pd.DataFrame(tracks)
tracks_df.to_csv('tracks_chartmetric.csv', index=False)